In [1]:
import simtk.openmm.app
import simtk.openmm
import simtk.unit as unit
import configparser
import numpy as np
import itertools
import scipy.spatial.distance as sdist
import os
import pdbfixer
import pandas
import subprocess
import nose

In [2]:
import openna

The location is /mnt/f/linux/script/openna
4.184


In [3]:
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(openna.xml)
s=forcefield.createSystem(top)

In [4]:
rna=openna.RNA.fromCoarsePDB('clean.pdb')
rna.periodic=False
rna.parseConfigurationFile(configuration_file=f'./openna.conf')

In [5]:
import forceTerm
all_forces = dict(Backbone=forceTerm.Backbone,
                 Angle=forceTerm.Angle)

for each_one in all_forces:
    force_now = all_forces[each_one](rna)
    s.addForce(force_now)

In [7]:
import numpy as np
temperature=300 * simtk.openmm.unit.kelvin
#platform_name='CUDA'

platform_name='Reference'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top, s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

250.71360500414966


In [8]:
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

TotalEnergy 250.713605 kJ/mol


In [18]:
energies = {}
for force_name, force in all_forces.items():
    force = force(rna)
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in all_forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

Backbone 0.0 kJ/mol
Angle 250.713605 kJ/mol


In [20]:
import sys
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
pdb_reporter=simtk.openmm.app.PDBReporter(f'output.pdb', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(energy_reporter)
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(50000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,79.77808435318452,324.160551730406
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,79.77808435318452,324.160551730406
2000,3.999999999999781,89.0014065131916,277.84906394133264
2000,3.999999999999781,89.0014065131916,277.84906394133264
3000,5.999999999999561,80.73959691396587,254.8935848451221
3000,5.999999999999561,80.73959691396587,254.8935848451221
4000,7.999999999999341,108.04860633700332,233.69132003050981
4000,7.999999999999341,108.04860633700332,233.69132003050981
5000,10.000000000000009,112.44343698324391,355.23092389597684
5000,10.000000000000009,112.44343698324391,355.23092389597684
6000,12.000000000000677,69.6918323481296,274.3551863709137
6000,12.000000000000677,69.6918323481296,274.3551863709137
7000,14.000000000001345,75.64803258646683,301.94116990305304
7000,14.000000000001345,75.64803258646683,301.94116990305304
8000,16.00000000000201,64.